
# **Assignment 1:MLP for Image Classification:Connectionist AI**

Imports and Data loading

In [93]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

(x_train, y_train), (x_test, y_test) = mnist.load_data()


Extracting labels

In [94]:
total_labels = len(np.unique(y_train))
print(f"labels:{np.unique(y_train)}")

labels:[0 1 2 3 4 5 6 7 8 9]


One-Hot encoding for classification

In [95]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Data preprocessing

In [96]:
input_size = x_train.shape[1]**2
x_train = np.reshape(x_train, (-1,input_size))
x_train = x_train.astype('float32') / 255

x_test = np.reshape(x_test, (-1,input_size))
x_test = x_test.astype('float32') / 255

print(f"x_train:{x_train.shape}")
print(f"x_test:{x_test.shape}")

x_train:(60000, 784)
x_test:(10000, 784)


Hyperparams

In [97]:
batch_size = 64
hidden_units = 256
dropout = 0.35

The Traditional MLP model

In [98]:
model = Sequential()

model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(total_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_190"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_358 (Dense)           (None, 256)               200960    
                                                                 
 activation_3 (Activation)   (None, 256)               0         
                                                                 
 dropout_175 (Dropout)       (None, 256)               0         
                                                                 
 dense_359 (Dense)           (None, 256)               65792     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_176 (Dropout)       (None, 256)               0         
                                                                 
 dense_360 (Dense)           (None, 10)             

MLP Model Training

In [100]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
938/938 [==============================] - 7s 6ms/step - loss: 0.3203 - accuracy: 0.9038
Epoch 2/10
938/938 [==============================] - 4s 5ms/step - loss: 0.1529 - accuracy: 0.9537
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1183 - accuracy: 0.9636
Epoch 4/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0986 - accuracy: 0.9702
Epoch 5/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0879 - accuracy: 0.9722
Epoch 6/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0833 - accuracy: 0.9739
Epoch 7/10
938/938 [==============================] - 5s 6ms/step - loss: 0.0707 - accuracy: 0.9775
Epoch 8/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0675 - accuracy: 0.9786
Epoch 9/10
938/938 [==============================] - 5s 6ms/step - loss: 0.0642 - accuracy: 0.9802
Epoch 10/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0591 - accuracy: 0.9812

MLP model evaluation

In [ ]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print(f"Accuracy: {100.0 * acc}%")

Accuracy: 98.2%


---
## Traditional MLP Accuracy: 98.2%
---

## MLP-Mixer model

Arguments(HyperParameters)

In [ ]:
patch_size = 4
num_patches = (28 // patch_size) ** 2
# print( (input_size // patch_size) ** 2)
hidden_units=64
learning_speed=0.05
# print(num_patches)

Imports

In [ ]:
from tensorflow.keras.layers import LayerNormalization, Layer, Input, GlobalAveragePooling1D, Dropout
from tensorflow.keras import Model
import tensorflow.keras as keras
from tensorflow import transpose,reshape,shape
from tensorflow.image import extract_patches

The Mixer Layer

In [ ]:
class MixerLayer(Layer):
    def __init__(self, num_patches, hidden_units, dropout, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.mlp1 = Sequential(
            [
                Dense(units=num_patches, activation="gelu"),
                Dense(units=num_patches),
                Dropout(rate=dropout),
            ]
        )
        self.mlp2 = Sequential(
            [
                Dense(units=num_patches, activation="gelu"),
                Dense(units=hidden_units),
                Dropout(rate=dropout),
            ]
        )
        self.normalize = LayerNormalization(epsilon=1e-6)

    def build(self, input_shape):
        return super().build(input_shape)

    def call(self, inputs):
        x = self.normalize(inputs)
        x_channels = transpose(x, perm=(0, 2, 1))
        mlp1_outputs = self.mlp1(x_channels)
        mlp1_outputs = transpose(mlp1_outputs, perm=(0, 2, 1))
        x = mlp1_outputs + inputs
        x_patches = self.normalize(x)
        mlp2_outputs = self.mlp2(x_patches)
        x = x + mlp2_outputs
        return x


The Patch extraction Layer

In [ ]:
class PatchesLayer(Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, x):
        patches = extract_patches(images=x, sizes=[1, self.patch_size, self.patch_size, 1], strides=[1, self.patch_size, self.patch_size, 1], rates=[1, 1, 1, 1], padding='VALID')
        batch_size = shape(patches)[0]
        num_patches = shape(patches)[1] * shape(patches)[2]
        patch_dim =shape(patches)[3]
        output = reshape(patches, (batch_size, num_patches, patch_dim))
        return output

Model Maker

In [88]:
def model_maker(blocks):
    #The input layer
    inputs = Input(shape=(28,28,1))
    #The patch extraction layer
    patches = PatchesLayer(patch_size)(inputs)
    x = Dense(units=hidden_units)(patches)
    x = blocks(x)
    #The average pooling layer
    condensed_input = GlobalAveragePooling1D()(x)
    condensed_input = Dropout(rate=dropout)(condensed_input)
    #The Output layer
    logits = Dense(total_labels,activation='softmax')(condensed_input)
    return Model(inputs=inputs, outputs=logits)

Building of the model using the Patch and

In [89]:
mixer_blocks = Sequential(
    [MixerLayer(num_patches, hidden_units, dropout) for _ in range(4)]
)
learning_rate = 0.005
mixer_model = model_maker(mixer_blocks)


Loading of dataset for the Mixer Model

In [ ]:
(x_train_res, y_train_res), (x_test_res, y_test_res) = mnist.load_data()
x_train_res = x_train_res.reshape((-1, 28, 28, 1))
x_test_res = x_test_res.reshape((-1, 28, 28, 1))
x_train_res = x_train_res.astype('float32') / 255
x_test_res = x_test_res.astype('float32') / 255

In [ ]:
mixer_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 patches_16 (Patches)        (None, 49, 16)            0         
                                                                 
 dense_320 (Dense)           (None, 49, 64)            1088      
                                                                 
 sequential_171 (Sequential  (None, 49, 64)            45652     
 )                                                               
                                                                 
 global_average_pooling1d_2  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_156 (Dropout)       (None, 64)                0   

Mixer Model Training

In [ ]:
mixer_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
mixer_model.fit(x_train_res, y_train_res, epochs=10, batch_size=64)

Epoch 1/10
938/938 [==============================] - 27s 19ms/step - loss: 0.6319 - accuracy: 0.7923
Epoch 2/10
938/938 [==============================] - 18s 19ms/step - loss: 0.2753 - accuracy: 0.9158
Epoch 3/10
938/938 [==============================] - 19s 20ms/step - loss: 0.2051 - accuracy: 0.9384
Epoch 4/10
938/938 [==============================] - 17s 18ms/step - loss: 0.1681 - accuracy: 0.9492
Epoch 5/10
938/938 [==============================] - 22s 23ms/step - loss: 0.1494 - accuracy: 0.9548
Epoch 6/10
938/938 [==============================] - 18s 19ms/step - loss: 0.1337 - accuracy: 0.9593
Epoch 7/10
938/938 [==============================] - 19s 20ms/step - loss: 0.1203 - accuracy: 0.9627
Epoch 8/10
938/938 [==============================] - 18s 19ms/step - loss: 0.1141 - accuracy: 0.9656
Epoch 9/10
938/938 [==============================] - 18s 19ms/step - loss: 0.1053 - accuracy: 0.9677
Epoch 10/10
938/938 [==============================] - 21s 22ms/step - loss: 0.099

Mixer Model Evaluation

In [ ]:
_, acc_mixer = mixer_model.evaluate(x_test_res, y_test_res, batch_size=64, verbose=0)
print(f"Mixer MLP Accuracy: {100*acc}%")

Mixer MLP Accuracy: 97.4%


---
## Mixer-MLP accuracy: 97.4%
---

Looking at the models from various perspectives it is clear that the Traditional MLP outperforms the MLP mixer model by a difference of 0.9%.
The reasons being the overall complexity of the task and the complexity of the models: where Traditional MLP is a much simpler model and would perform good on simple tasks the Mixer model is not designed for such low level tasks. And would thus beat the Traditional model on other tasks. Also it can be pretrained to further diversify it's usage and improve overall accuracy.

##References
- https://www.analyticsvidhya.com/blog/2020/12/mlp-multilayer-perceptron-simple-overview/
- https://www.v7labs.com/blog/neural-networks-activation-functions
- https://www.baeldung.com/cs/learning-rate-batch-size#:~:text=Batch%20size%20defines%20the%20number,training%20set%20in%20one%20epoch.
- https://arxiv.org/abs/2105.03404v2
- https://sh-tsang.medium.com/review-resmlp-feedforward-networks-for-image-classification-with-data-efficient-training-4eeb1eb5efa6
- https://stats.stackexchange.com/questions/162988/why-sigmoid-function-instead-of-anything-else/318209#318209